In [ ]:
!pip install -q gdown
%cd {HOME}
!gdown '1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-'

/content
Downloading...
From: https://drive.google.com/uc?id=1pz68D1Gsx80MoPg-_q-IbEdESEmyVLm-
To: /content/vehicle-counting.mp4
100% 35.3M/35.3M [00:00<00:00, 165MB/s]


In [1]:
import cv2
!pip install ultralytics
import ultralytics
ultralytics.checks()

!pip install supervision
import supervision as sv

Ultralytics YOLOv8.2.74 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.6/78.2 GB disk)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.0 MB/s eta 0:00:00


In [2]:
from ultralytics import YOLO
model = YOLO("yolov8x.pt")
model.fuse()

100%|██████████| 131M/131M [00:00<00:00, 412MB/s] 


YOLOv8x summary (fused): 268 layers, 68,200,608 parameters, 0 gradients, 257.8 GFLOPs


In [3]:
CLASS_NAMES_DICT = model.model.names

selected_classes = [2,3,5,7]

In [4]:
def box_anotator(frame, detections):

        for detection_idx in range(len(detections)):

            x1, y1, x2, y2 = detections.xyxy[detection_idx].astype(int)
            cv2.rectangle(img = frame,pt1 = (x1, y1),pt2 = (x2, y2),color = (0, 255, 255),thickness= 2,)

        return frame


In [79]:
def crowd_calculator(frame, detections, height, width):

    in_car = 0
    out_car = 0

    for detection_idx in range(len(detections)):

        x1, y1, x2, y2 = detections.xyxy[detection_idx].astype(int)

        if y1 > height and x1 < width *1/2 :
           in_car += 1

        if y1< height  and x1 > width *1/2 :
           out_car += 1


        #make and plot the line counter
    cv2.line(frame,(0, height), (width, height), (0,0, 255), 2)


    cv2.putText(frame, f' in:{in_car}', (0,height -10), cv2.FONT_HERSHEY_SIMPLEX ,  1, (255,0,0), 2, cv2.LINE_AA)
    cv2.putText(frame, f'out: {out_car}', (0, height + 30), cv2.FONT_ITALIC ,  1, (255,0,0), 2, cv2.LOCAL_OPTIM_SIGMA)

    return frame

In [13]:
video_path = '/content/sample_video.mp4'
import numpy as np

In [80]:
# create frame generator
generator = sv.get_video_frames_generator(video_path)
# create instance of BoxAnnotator
box_annotator = sv.BoxAnnotator()
# acquire first video frame
iterator = iter(generator)
frame = next(iterator)
# model prediction on single frame and conversion to supervision Detections
results = model(frame, verbose=False)[0]

# convert to Detections
detections = sv.Detections.from_ultralytics(results)
# only consider class id from selected_classes define above
detections = detections[np.isin(detections.class_id, selected_classes)]

# format custom labels
labels = [
    f"{CLASS_NAMES_DICT[class_id]} {confidence:0.2f}"
    for confidence, class_id in zip(detections.confidence, detections.class_id)


]

cap = cv2.VideoCapture(video_path)
# cap.set(cv2.CAP_PROP_POS_FRAMES, 200)  # Replace 50 with your desired frame number

ret, frame = cap.read()

height, width, channels = frame.shape

height  = int(height-(height*1/3))
width = int(width)



frame = crowd_calculator(frame, detections, height, width)
# annotate and display frame
anotated_frame=box_annotator.annotate(scene=frame, detections=detections)

%matplotlib inline
sv.plot_image(anotated_frame, (16,16))

UnboundLocalError: local variable 'out_car' referenced before assignment

In [77]:
video_path = '/content/sample_video.mp4'

cap= cv2.VideoCapture(video_path)
ret, frame = cap.read()

height, width, channels = frame.shape
print(frame.shape)

(720, 1280, 3)


In [78]:
video_path = '/content/sample_video.mp4'

import numpy as np

byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)
cap = cv2.VideoCapture(video_path)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'MJPG')
out = cv2.VideoWriter('output.mp4', fourcc, 30.0, (1280,  720))

while cap.isOpened():
    ret, frame = cap.read()
    # Break the loop if the end of the video is reached
    if not ret:
        break

    height, width, channels = frame.shape

    results = model(frame, verbose = False)[0]

    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id, selected_classes)]

    detections = byte_tracker.update_with_detections(detections)
    frame = box_anotator(frame, detections)


    height  = int(height-(height*1/3))
    width = int(width)

    frame = crowd_calculator(frame, detections, height, width)

    out.write(frame)

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.


In [ ]:
import numpy as np

class LineZone:
  def __init__(self, start, end):
    self.vector = np.array(end) - np.array(start)
    self.in_count = 0
    self.out_count = 0

  def is_in_zone(self, point):
    # Simplified calculation using cross product
    return np.cross(self.vector, point - self.vector.start) > 0

  def trigger(self, detections, tracker_id):
    crossed_in = np.zeros(len(detections))
    crossed_out = np.zeros(len(detections))
    for i, detection in enumerate(detections):
      if self.is_in_zone(detection[tracker_id]):
        crossed_out[i] = 1
      else:
        crossed_in[i] = 1
    self.in_count += crossed_in.sum()
    self.out_count += crossed_out.sum()
    return crossed_in, crossed_out


In [ ]:
models = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Support Vector Machines','Desision Tree '],
    'R-squared Score': [lin_r_squre*100, randome_forest*100, svr_r_squre*100, tree_r_squre*100]})
models.sort_values(by='R-squared Score', ascending=False)